# Virtualizing a processor

When we say "virtualizing" what we mean is actually adding a layer of indirection between the target (in this case the CPU) and the user (the process). In order to do this, we will need a way of pausing the execution of a process while another process runs and then resuming execution later without the process having to be aware of this pause which means we need to capture all of the state that is relevant to a process in one or more data structures we can use to hold them when the process is not active.